这是2025秋北京大学CV的Final Project，我们选择对当前的Diffusion Illusion进行改进，以期待能够产出更有创造力的作品。这个juypter notebook的目标是创造对于两张图片，更加细化的旋转能够产出不同的图片，甚至产出故事的效果。

以下的大多数编程块都是直接由项目粘贴过来，而如果有自己修改的部分，我会做出标识以供debug

In [ ]:
%%bash
if [ ! -d ".git" ]; then 
    rm -rf * .*; #Get rid of Colab's default junk files
    git clone -b master https://github.com/dreaveler/Diffusion-Illusions.git .
fi

In [ ]:
%pip install --upgrade -r requirements.txt
%pip install rp --upgrade
# You may need to restart the runtime after installing these
# I'm not sure why this helps, but all sorts of weird random errors pop up in Colab if you don't

In [ ]:
import numpy as np
import rp
import torch
import torchvision.transforms.functional as TF
from torchvision.transforms import InterpolationMode
import torch.nn as nn
import source.stable_diffusion as sd
from easydict import EasyDict
from source.learnable_textures import LearnableImageFourier
from source.stable_diffusion_labels import NegativeLabel
from itertools import chain
import time

下面会是第一个需要修改的部分 关于prompts 我暂时的想法是两张图片的旋转离散到8个角度去，这样可以保证中间叠加过后的图形是可控的

In [ ]:
example_prompts = rp.load_yaml_file('source/example_prompts.yaml')
print('Available example prompts:', ', '.join(example_prompts))

#These prompts are all strings - you can replace them with whatever you want! By default it lets you choose from example prompts
prompt_a = "A lion"
prompt_b = "A cat"
prompt_c = "miku"
prompt_d = "dog"
prompt_e = "flower"
prompt_f = "Kobe Bryant"
prompt_g = "snake"
prompt_h = "Harry Potter"

negative_prompt = ''

print()
print('Negative prompt:',repr(negative_prompt))
print()
print('Chosen prompts:')
print('    prompt_a =', repr(prompt_a))
print('    prompt_b =', repr(prompt_b))
print('    prompt_c =', repr(prompt_c))
print('    prompt_d =', repr(prompt_d))
print('    prompt_e =', repr(prompt_e))
print('    prompt_f =', repr(prompt_f))
print('    prompt_g =', repr(prompt_g))
print('    prompt_h =', repr(prompt_h))

In [ ]:
if 's' not in dir():
    model_name="CompVis/stable-diffusion-v1-4"
    gpu='cuda:0'
    s=sd.StableDiffusion(gpu,model_name)
device=s.device

下面这部分的改动同上 由于prompt

In [ ]:
label_a = NegativeLabel(prompt_a,negative_prompt)
label_b = NegativeLabel(prompt_b,negative_prompt)
label_c = NegativeLabel(prompt_c,negative_prompt)
label_d = NegativeLabel(prompt_d,negative_prompt)
label_e = NegativeLabel(prompt_e,negative_prompt)
label_f = NegativeLabel(prompt_f,negative_prompt)
label_g = NegativeLabel(prompt_g,negative_prompt)
label_h = NegativeLabel(prompt_h,negative_prompt)

In [ ]:
#Image Parametrization and Initialization (this section takes vram)

#Select Learnable Image Size (this has big VRAM implications!):
#Note: We use implicit neural representations for better image quality
#They're previously used in our paper "TRITON: Neural Neural Textures make Sim2Real Consistent" (see tritonpaper.github.io)
# ... and that representation is based on Fourier Feature Networks (see bmild.github.io/fourfeat)
learnable_image_maker = lambda: LearnableImageFourier(height=256, width=256, hidden_dim=256, num_features=128).to(s.device); SIZE=256
# learnable_image_maker = lambda: LearnableImageFourier(height=512,width=512,num_features=256,hidden_dim=256,scale=20).to(s.device);SIZE=512

bottom_image=learnable_image_maker()
top_image=learnable_image_maker()

下面这部分也同样需要修改

In [ ]:
brightness=3

CLEAN_MODE = True # If it's False, we augment the images by randomly simulating how good a random printer might be when making the overlays...

def simulate_overlay(bottom, top):
    if CLEAN_MODE:
        exp=1
        brightness=3
        black=0
    else:
        exp=rp.random_float(.5,1)
        brightness=rp.random_float(1,5)
        black=rp.random_float(0,.5)
        bottom=rp.blend(bottom,black,rp.random_float())
        top=rp.blend(top,black,rp.random_float())
    return (bottom**exp * top**exp * brightness).clamp(0,99).tanh()

def rotate45(image,k):
    return TF.rotate(image, angle=45*k, interpolation=InterpolationMode.BILINEAR, expand=False)


learnable_image_a=lambda: simulate_overlay(bottom_image(), rotate45(top_image(),k=0))
learnable_image_b=lambda: simulate_overlay(bottom_image(), rotate45(top_image(),k=1))
learnable_image_c=lambda: simulate_overlay(bottom_image(), rotate45(top_image(),k=2))
learnable_image_d=lambda: simulate_overlay(bottom_image(), rotate45(top_image(),k=3))
learnable_image_e=lambda: simulate_overlay(bottom_image(), rotate45(top_image(),k=4))
learnable_image_f=lambda: simulate_overlay(bottom_image(), rotate45(top_image(),k=5))
learnable_image_g=lambda: simulate_overlay(bottom_image(), rotate45(top_image(),k=6))
learnable_image_h=lambda: simulate_overlay(bottom_image(), rotate45(top_image(),k=7))
learnable_image_names=['image_0','image_45','image_90','image_135','image_180','image_225','image_270','image_315']



params=chain(
    bottom_image.parameters(),
    top_image.parameters(),
)
optim=torch.optim.SGD(params,lr=1e-4)

下面的代码块也需要修改

In [ ]:
nums=[0,1,2,3,4,5,6,7]

#Uncommenting one of the lines will disable some of the prompts, in case you don't want to use all four for some reason (like the Summer/Winter example)
# nums=[0  ,2,3]
# nums=[    2  ]
# nums=[0,1,2]
# nums=[1]
# nums=[0,1]
# nums=[0,2]


labels=[label_a,label_b,label_c,label_d,label_e,label_f,label_g,label_h]
learnable_images=[learnable_image_a,learnable_image_b,learnable_image_c,learnable_image_d,
                   learnable_image_e,learnable_image_f,learnable_image_g,learnable_image_h]
learnable_image_names=[name for name in learnable_image_names]

#The weight coefficients for each prompt. For example, if we have [0,1,2,1], then prompt_w will provide no influence and prompt_y will have 1/2 the total influence
weights=[1,1,1,1,1,1,1,1]

labels=[labels[i] for i in nums]
learnable_images=[learnable_images[i] for i in nums]
learnable_image_names=[learnable_image_names[i] for i in nums]
weights=[weights[i] for i in nums]

weights=rp.as_numpy_array(weights)
weights=weights/weights.sum()
weights=weights*len(weights)

In [ ]:
#For saving a timelapse
ims=[]

In [ ]:
def get_display_image():
    return rp.tiled_images(
        [
            *[rp.as_numpy_image(image()) for image in learnable_images],
            rp.as_numpy_image(bottom_image()),
            rp.as_numpy_image(top_image()),
        ],
        length=len(learnable_images),
        border_thickness=0,
    )

In [ ]:
NUM_ITER=10000

#Set the minimum and maximum noise timesteps for the dream loss (aka score distillation loss)
s.max_step=MAX_STEP=990
s.min_step=MIN_STEP=10 

display_eta=rp.eta(NUM_ITER, title='Status: ')

DISPLAY_INTERVAL = 200

print('Every %i iterations we display an image in the form [[%s], [bottom_image, top_image]] where'% (DISPLAY_INTERVAL, ', '.join(learnable_image_names)))
image_defs={
    'image_0': 'bottom_image * rotate45(top_image, k=0)',
    'image_45': 'bottom_image * rotate45(top_image, k=1)',
    'image_90': 'bottom_image * rotate45(top_image, k=2)',
    'image_135': 'bottom_image * rotate45(top_image, k=3)',
    'image_180': 'bottom_image * rotate45(top_image, k=4)',
    'image_225': 'bottom_image * rotate45(top_image, k=5)',
    'image_270': 'bottom_image * rotate45(top_image, k=6)',
    'image_315': 'bottom_image * rotate45(top_image, k=7)',
}
for name in learnable_image_names:
    print(f'    {name} = {image_defs[name]}')
print()
print('Interrupt the kernel at any time to return the currently displayed image')
print('You can run this cell again to resume training later on')
print()
print('Please expect this to take hours to get good images (especially on the slower Colab GPU\'s! The longer you wait the better they\'ll be')

try:
    for iter_num in range(NUM_ITER):
        display_eta(iter_num) #Print the remaining time

        preds=[]
        for label,learnable_image,weight in rp.random_batch(list(zip(labels,learnable_images,weights)), batch_size=1):
            pred=s.train_step(
                label.embedding,
                learnable_image()[None],

                #PRESETS (uncomment one):
                noise_coef=.1*weight,guidance_scale=60,#10
                # noise_coef=0,image_coef=-.01,guidance_scale=50,
                # noise_coef=0,image_coef=-.005,guidance_scale=50,
                # noise_coef=.1,image_coef=-.010,guidance_scale=50,
                # noise_coef=.1,image_coef=-.005,guidance_scale=50,
                # noise_coef=.1*weight, image_coef=-.005*weight, guidance_scale=50,
            )
            preds+=list(pred)

        with torch.no_grad():
            if iter_num and not iter_num%(DISPLAY_INTERVAL*50):
                #Wipe the slate every 50 displays so they don't get cut off
                from IPython.display import clear_output
                clear_output()

            if not iter_num%DISPLAY_INTERVAL:
                im = get_display_image()
                ims.append(im)
                rp.display_image(im)

        optim.step()
        optim.zero_grad()
except KeyboardInterrupt:
    print()
    print('Interrupted early at iteration %i'%iter_num)
    im = get_display_image()
    ims.append(im)
    rp.display_image(im)

In [ ]:
print('Bottom image:')
rp.display_image(rp.as_numpy_image(factor_base()))

print('Top image:')
rp.display_image(rp.as_numpy_image(factor_rotator()))

In [ ]:
def save_run(name):
    folder="untracked/rotator_multiplier_runs/%s"%name
    if rp.path_exists(folder):
        folder+='_%i'%time.time()
    rp.make_directory(folder)
    ims_names=['ims_%04i.png'%i for i in range(len(ims))]
    with rp.SetCurrentDirectoryTemporarily(folder):
        rp.save_images(ims,ims_names,show_progress=True)
    print()
    print('Saved timelapse to folder:',repr(folder))
    
save_run('untitled') #You can give it a good custom name if you want!